### **Web Scraping:** 
**El web scraping es el proceso de extracción automatizada de datos de sitios web. Utiliza programas de software para recorrer páginas web, analizar su estructura y extraer información específica de ellas. Este proceso puede incluir la descarga de contenido web, el análisis de HTML para identificar patrones de datos y la extracción de datos relevantes según ciertos criterios, como etiquetas HTML específicas, clases CSS o ubicaciones en la página. El objetivo del web scraping es obtener datos de manera eficiente y automatizada para su posterior análisis, almacenamiento o uso en otras aplicaciones.**

#### Importamos librerias necesarias para nuestro trabajo

In [1]:
import pandas as pd  # Importar la librería pandas para manipulación y análisis de datos
import os  # Importar el módulo os para operaciones relacionadas con el sistema operativo
import re  # Importar el módulo re para operaciones de expresiones regulares
import requests  # Importar el módulo requests para realizar solicitudes HTTP
from bs4 import BeautifulSoup  # Importar BeautifulSoup desde el módulo bs4 para analizar HTML
import fastparquet  # Importar la biblioteca fastparquet para leer y escribir archivos Parquet

#### Primer WebScraping: página web de **Comisión de Taxis y Limusinas de Nueva York (TLC)**

Registros de viajes en taxis amarillos y verdes, incluye información sobre fechas, horas, ubicaciones, distancias, tarifas y tipos de pago. Estos datos fueron recopilados por proveedores tecnológicos autorizados y presentados a la Comisión de Taxis y Limusinas de Nueva York (TLC). https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [2]:
def create_directory(directory):
    """Crea un directorio si no existe."""
    if not os.path.exists(directory):  # Verificar si el directorio no existe
        os.makedirs(directory)  # Crear el directorio si no existe

def download_file(url, filename):
    """Descarga un archivo desde la URL y lo guarda con el nombre especificado."""
    response = requests.get(url)  # Realizar una solicitud GET a la URL especificada
    if response.status_code == 200:  # Verificar si la solicitud fue exitosa (código de estado 200)
        with open(filename, 'wb') as file:  # Abrir el archivo en modo binario para escritura
            file.write(response.content)  # Escribir el contenido de la respuesta en el archivo
        print(f"Archivo descargado: {filename}")  # Imprimir un mensaje indicando que el archivo ha sido descargado

# Crear carpeta de destino si no existe
carpeta_destino = "../0-DataSets/1-DatosScraping"
create_directory(carpeta_destino)

# Obtener la lista de archivos en la carpeta de destino
archivos_locales = os.listdir(carpeta_destino)

# Generar enlaces para los meses del 01 al 12
meses = []
for nums in range(1,13):  # Iterar sobre números del 1 al 12
    if nums < 10:
        meses.append('0' + str(nums))  # Agregar un cero delante si el número es menor que 10
    else:
        meses.append(str(nums))  # Convertir el número a cadena y agregarlo sin modificar si es mayor o igual a 10

# URL de la página web
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Realizar la solicitud HTTP a la página
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Parsear el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar todos los enlaces que contienen 'yellow' o 'green' y tienen el formato esperado para 2023
    links_pagina = {a['href'] for a in soup.find_all('a', href=True) if re.match(r'^.*(yellow|green)_tripdata_(2023|2024)-\d{2}\.parquet$', a['href'])}

    # Combinar enlaces generados dinámicamente con los obtenidos de la página
    links_totales = links_pagina.copy()  # Crear una copia para mantener los enlaces originales

    links_totales.update({
        f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{año}-{mes}.parquet" 
        for año in ['2023', '2024']  # Iterar sobre los años
        for mes in meses
    })
    links_totales.update({
        f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{año}-{mes}.parquet" 
        for año in ['2023', '2024']  # Iterar sobre los años
        for mes in meses
    })

    # Realizar la descarga de los archivos
    for link in sorted(links_totales):  # Iterar sobre los enlaces ordenados alfabéticamente
        nombre_archivo = link.split("/")[-1]  # Obtener el nombre del archivo desde la URL
        archivo_destino = os.path.join(carpeta_destino, nombre_archivo)  # Construir la ruta completa del archivo destino

        if nombre_archivo not in archivos_locales:  # Verificar si el archivo no existe localmente
            download_file(link, archivo_destino)  # Descargar el archivo desde el enlace proporcionado
        else:
            print(f"El archivo {nombre_archivo} ya existe localmente.")  # Imprimir un mensaje indicando que el archivo ya existe
else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")  # Imprimir un mensaje de error si la solicitud no fue exitosa


Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-01.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-02.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-03.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-04.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-05.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-06.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-07.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-08.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-09.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-10.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-11.parquet
Archivo descargado: ../0-DataSets/1-DatosScraping\green_tripdata_2023-12.parquet
Archivo descargado: ../0-Dat

#### Realizamos un `ETL` de los `archivos descargados` para tener un Dataset según proposito del proyecto.

###### **`DICCIONARIO`** <br>* 'VendorID': Un código que indica el proveedor de LPEP que proporcionó el registro.<br>1= Tecnologías móviles creativas, LLC;<br> 2= ​​VeriFone Inc.<br>* 'lpep_pickup_datetime': La fecha y hora en que se activó el medidor.<br>* 'lpep_dropoff_datetime': La fecha y hora en que se desconectó el medidor.<br>* 'store_and_fwd_flag': Esta bandera indica si el registro de viaje se mantuvo en el vehículo.memoria antes de enviar al proveedor, también conocido como "almacenar y reenviar",porque el vehículo no tenía conexión con el servidor.<br>Y = viaje de almacenamiento y avance<br>N= no es un viaje de ida y vuelta<br>* 'RatecodeID': El código de tarifa final vigente al finalizar el viaje.<br>1= Tarifa estándar<br>2=JFK<br>3=Newark<br>4=Nasáu o Westchester<br>5=Tarifa negociada<br>6=viaje en grupo<br>* 'PULocationID': Zona de Taxi TLC en la que estaba activado el taxímetro<br>* 'DOLocationID': Zona de Taxi TLC en la que se desactivó el taxímetro<br>* 'passenger_count': El número de pasajeros en el vehículo. Este es un valor ingresado por el conductor.<br>* 'trip_distance': La distancia del viaje transcurrido en millas reportadas por el taxímetro.<br>* 'fare_amount': La tarifa de tiempo y distancia calculada por el taxímetro.<br>* 'extra': Extras y recargos varios. Actualmente esto sólo incluye los cargos de $0,50 y $1 por hora pico y por noche.<br>* 'mta_tax': Impuesto MTA de $0.50 que se activa automáticamente según el medidor tasa en uso.<br>* 'tip_amount': Monto de la propina: este campo se completa automáticamente para tarjetas de crédito. consejos. Las propinas en efectivo no están incluidas.<br>* 'tolls_amount': Importe total de todos los peajes pagados en el viaje.<br>* 'ehail_fee': ???<br>* 'improvement_surcharge': Recargo por mejora de $0.30 aplicado en viajes granizados en la bandera gota. El recargo por mejora comenzó a cobrarse en 2015.<br>* 'total_amount': El importe total cobrado a los pasajeros. No incluye propinas en efectivo.<br>* 'payment_type': Un código numérico que indica cómo el pasajero pagó el viaje.<br>1= tarjeta de crédito<br>2= ​​Efectivo<br>3= Sin cargo<br>4= Disputa<br>5= Desconocido<br>6= Viaje anulado<br>* 'trip_type': Un código que indica si el viaje fue un llamado callejero o un despacho.que se asigna automáticamente según la tarifa medida en uso, pero puede ser modificado por el conductor.<br>1= Granizo callejero<br>2= ​​Despacho<br>* 'congestion_surcharge':"
